<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/conv_specification_example/tests/conv_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install onnx onnxruntime

In [2]:
from onnx import TensorProto
from onnx.helper import (
    make_model, make_node, make_graph,
    make_tensor_value_info)
from onnx.checker import check_model

In [8]:
# @title Standard max pooling (1 channel)
import numpy as np
from onnx import *
import onnxruntime

# Create inputs
x = helper.make_tensor_value_info('x', TensorProto.DOUBLE, [1, 1, 8, 8])

# Create a node (MaxPool) with input/outputs
node_def = make_node(
    'MaxPool', # node name
    ['x'], # inputs
    ['y', 'Indices'], # outputs
    ceil_mode = 0,
    dilations=[1,1],
    kernel_shape=[3,3],
    pads=[1, 0, 1, 0],
    strides=[1, 1],
    auto_pad='NOTSET',
#    group=1, # Standard convolution
)

# Create the graph
graph_def = make_graph(
    [node_def],
    'test-maxpool',
    [x],
    [helper.make_tensor_value_info('y', TensorProto.DOUBLE, [1, 1, 8, 6]), 
     helper.make_tensor_value_info('Indices', TensorProto.INT64, [1, 1, 8, 6])],
)

onnx_model = make_model(graph_def)

# Let's freeze the opset.
del onnx_model.opset_import[:]
opset = onnx_model.opset_import.add()
opset.domain = ''
opset.version = 15
onnx_model.ir_version = 8

# Verify the model
check_model(onnx_model)

# Print a human-readable representation of the graph
print(onnx.helper.printable_graph(onnx_model.graph))

# Do inference
sess = onnxruntime.InferenceSession(onnx_model.SerializeToString(),
                        providers=["CPUExecutionProvider"])

# Initialize tensors
#x = numpy.ones((1, 1, 8, 8), dtype=numpy.float32)

x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))

y = sess.run(None, {'x': x})[0]
Indices = sess.run(None, {'x': x})[1]

print("X shape:", x.shape)
print("X:", x)
print("Y shape:", y.shape)
print("Y:", y)
print("Indices shape:", Indices.shape)
print("Indices:", Indices)


graph test-maxpool (
  %x[DOUBLE, 1x1x8x8]
) {
  %y, %Indices = MaxPool[auto_pad = 'NOTSET', ceil_mode = 0, dilations = [1, 1], kernel_shape = [3, 3], pads = [1, 0, 1, 0], strides = [1, 1]](%x)
  return %y, %Indices
}
X shape: (1, 1, 8, 8)
X: [[[[ 1.72165569  6.84315922  3.376594    3.49528318  4.46102348
     1.99059382  1.43579812  0.52530497]
   [ 5.52246771  2.15325489  1.38958441 -0.90843431  3.29150389
     3.88003463 -0.53965053  0.18804484]
   [ 5.80536068  4.86458125  1.92312541  0.15635311  4.50913422
     2.60292532 -0.84740939  1.50776103]
   [ 2.79261426  4.13621027  5.6250972  -0.16892762  4.831916
     3.31882283  2.48772745  3.41486079]
   [ 2.69632676 -0.68805249  1.44033855  3.09276577  2.43350826
     4.71254399  3.8998928   5.63390124]
   [-0.66246716  3.41150814  2.97346869  1.06119413  5.10054163
     4.02036376  5.40287741  3.23663817]
   [-1.05639711  5.10054251  3.90693852  1.30658099  2.3367182
     3.33210516  3.44673094  1.31835211]
   [ 6.19178149  0.263495